In [26]:
import pandas as pd
import json

In [27]:
df = pd.read_csv('ncaa_results.csv')

In [28]:
regions = [team for team in df["Region Name"].unique() if team not in ["Final Four", "Championship"]]

In [29]:
df.loc[df['Year'] == 1985]

Year  Round  Region Number   Region Name  Seed  Score             Team  \
0   1985      1              1          West     1     83         St Johns   
1   1985      1              1          West     2     81              VCU   
2   1985      1              1          West     3     65         NC State   
3   1985      1              1          West     4     85             UNLV   
4   1985      1              1          West     5     58       Washington   
5   1985      1              1          West     6     75            Tulsa   
6   1985      1              1          West     7     50          Alabama   
7   1985      1              1          West     8     54             Iowa   
8   1985      1              2          East     1     68       Georgetown   
9   1985      1              2          East     2     65     Georgia Tech   
10  1985      1              2          East     3     76         Illinois   
11  1985      1              2          East     4     59  Loyola Illinois   
12  1985      1              2          East     5     85              SMU   
13  1985      1              2          East     6     67          Georgia   
14  1985      1              2          East     7     70         Syracuse   
15  1985      1              2          East     8     60           Temple   
16  1985      1              3       Midwest     1     96         Oklahoma   
17  1985      1              3       Midwest     2     67          Memphis   
18  1985      1              3       Midwest     3     75             Duke   
19  1985      1              3       Midwest     4     75          Ohio St   
20  1985      1              3       Midwest     5     78   Louisiana Tech   
21  1985      1              3       Midwest     6     53       Texas Tech   
22  1985      1              3       Midwest     7     70              UAB   
23  1985      1              3       Midwest     8     55              USC   
24  1985      1              4     Southeast     1     59         Michigan   
25  1985      1              4     Southeast     2     76   North Carolina   
26  1985      1              4     Southeast     3     49           Kansas   
27  1985      1              4     Southeast     4     55              LSU   
28  1985      1              4     Southeast     5     69         Maryland   
29  1985      1              4     Southeast     6     58           Purdue   
..   ...    ...            ...           ...   ...    ...              ...   
33  1985      2              1          West     2     59              VCU   
34  1985      2              1          West     3     86         NC State   
35  1985      2              1          West     4     61             UNLV   
36  1985      2              2          East     1     63       Georgetown   
37  1985      2              2          East     2     70     Georgia Tech   
38  1985      2              2          East     3     74         Illinois   
39  1985      2              2          East     4     70  Loyola Illinois   
40  1985      2              3       Midwest     1     75         Oklahoma   
41  1985      2              3       Midwest     2     67          Memphis   
42  1985      2              3       Midwest     3     73             Duke   
43  1985      2              3       Midwest     4     67          Ohio St   
44  1985      2              4     Southeast     1     55         Michigan   
45  1985      2              4     Southeast     2     60   North Carolina   
46  1985      2              4     Southeast     3     64           Kansas   
47  1985      2              4     Southeast    13     59             Navy   
48  1985      3              1          West     1     86         St Johns   
49  1985      3              1          West     7     55          Alabama   
50  1985      3              2          East     1     65       Georgetown   
51  1985      3              2          East     2     61     Georgia Tech   
52  1985      3           

In [30]:
def empty_tournament():
    ''' Generate an empty tournament '''
    result = []
    for round_size in [32, 16, 8, 4, 2, 1]:
        round_result = []
        for j in range(round_size):
            round_result.append([None, None])
        result.append(round_result)
    return result

In [31]:
def empty_tournament_by_region():
    result = {}
    for region in regions:
        region_result = []
        for round_size in [8, 4, 2, 1]:
            round_result = []
            for j in range(round_size):
                round_result.append([None, None])
            region_result.append(round_result)
        result[region] = region_result
    result['Final Four'] = [[[None, None], [None, None]]]
    result['Championship'] = [[[None, None]]]
    return result

In [32]:
empty_tournament_by_region()

{'Championship': [[[None, None]]],
 'East': [[[None, None],
   [None, None],
   [None, None],
   [None, None],
   [None, None],
   [None, None],
   [None, None],
   [None, None]],
  [[None, None], [None, None], [None, None], [None, None]],
  [[None, None], [None, None]],
  [[None, None]]],
 'Final Four': [[[None, None], [None, None]]],
 'Midwest': [[[None, None],
   [None, None],
   [None, None],
   [None, None],
   [None, None],
   [None, None],
   [None, None],
   [None, None]],
  [[None, None], [None, None], [None, None], [None, None]],
  [[None, None], [None, None]],
  [[None, None]]],
 'South': [[[None, None],
   [None, None],
   [None, None],
   [None, None],
   [None, None],
   [None, None],
   [None, None],
   [None, None]],
  [[None, None], [None, None], [None, None], [None, None]],
  [[None, None], [None, None]],
  [[None, None]]],
 'Southeast': [[[None, None],
   [None, None],
   [None, None],
   [None, None],
   [None, None],
   [None, None],
   [None, None],
   [None, None

In [34]:
def process_year(df, year):
    result = []
    year_df = df.loc[df['Year'] == year]
    for round_number in year_df.Round.unique():
        round_result = []
        round_df = year_df.loc[year_df["Round"] == round_number]
        for index, row in round_df.iterrows():
            round_result.append([
                {
                    "name": row["Team"],
                    "seed": row["Seed"],
                    "score": row["Score"],
                },
                {
                    "name": row["Team.1"],
                    "score": row["Score.1"],
                    "seed": row["Seed.1"]
                }
            ])
        result.append(round_result)
    return result

In [35]:
def process_year_by_region(df, year):
    ''' Process the year and split up the brackets by region '''
    result = {}
    year_df = df.loc[df['Year'] == year]
    for region in year_df["Region Name"].unique():
        region_df = year_df.loc[year_df['Region Name'] == region]
        region_result = []
        for round_number in region_df.Round.unique():
            round_result = []
            round_df = region_df.loc[region_df["Round"] == round_number]
            for index, row in round_df.iterrows():
                round_result.append([
                    {
                        "name": row["Team"],
                        "seed": row["Seed"],
                        "score": row["Score"],
                    },
                    {
                        "name": row["Team.1"],
                        "score": row["Score.1"],
                        "seed": row["Seed.1"]
                    }
                ])
            region_result.append(round_result)
        result[region] = region_result
    return result

In [36]:
def process_teams(df, year):
    ''' Get the teams and their associated rankings for each region '''
    result = {}
    # filter data frame by year and keep only first round where we see all the teams
    year_df = df.loc[df['Year'] == year]
    regions_df = year_df.loc[year_df['Round'] == 1]
    for region in regions:
        region_result = []
        region_df = regions_df.loc[regions_df['Region Name'] == region]
        for index, row in region_df.iterrows():
            region_result.append({
                "name": row['Team'],
                "seed": row['Seed']
            })
            region_result.append({
                "name": row['Team.1'],
                "seed": row['Seed.1']
            })
        region_result.sort(key=lambda k: k["seed"])
        result[region] = region_result
    return result

In [37]:
process_year_by_region(df, 1985)

{'Championship': [[[{'name': 'Georgetown', 'score': 64, 'seed': 1},
    {'name': 'Villanova', 'score': 66, 'seed': 8}]]],
 'East': [[[{'name': 'Georgetown', 'score': 68, 'seed': 1},
    {'name': 'Lehigh', 'score': 43, 'seed': 16}],
   [{'name': 'Georgia Tech', 'score': 65, 'seed': 2},
    {'name': 'Mercer', 'score': 58, 'seed': 15}],
   [{'name': 'Illinois', 'score': 76, 'seed': 3},
    {'name': 'Northeastern', 'score': 57, 'seed': 14}],
   [{'name': 'Loyola Illinois', 'score': 59, 'seed': 4},
    {'name': 'Iona', 'score': 58, 'seed': 13}],
   [{'name': 'SMU', 'score': 85, 'seed': 5},
    {'name': 'Old Dominion', 'score': 68, 'seed': 12}],
   [{'name': 'Georgia', 'score': 67, 'seed': 6},
    {'name': 'Wichita St', 'score': 59, 'seed': 11}],
   [{'name': 'Syracuse', 'score': 70, 'seed': 7},
    {'name': 'DePaul', 'score': 65, 'seed': 10}],
   [{'name': 'Temple', 'score': 60, 'seed': 8},
    {'name': 'Virginia Tech', 'score': 57, 'seed': 9}]],
  [[{'name': 'Georgetown', 'score': 63, 'see

In [38]:
def process_all_years(df, by_region=False):
    data = {}
    for year in range(1985, 2018):
        data[year] = process_year_by_region(df, year) if by_region else process_year(df, year)
#     data[2018] = empty_tournament_by_region() if by_region else empty_tournament()
    return data

In [39]:
with open('master.json', 'w') as outfile:
    tournaments = process_all_years(df)
    tournaments_by_region = process_all_years(df, True)
    data = {
        "tournaments": tournaments,
        "tournaments_by_region": tournaments_by_region
    }
    json.dump(data, outfile, indent=4)

In [85]:
with open('empty.json', 'w') as outfile:
    json.dump(empty_tournament(), outfile, indent=4, sort_keys=True)
with open('empty_by_region.json', 'w') as outfile:
    json.dump(empty_tournament_by_region(), outfile, indent=4, sort_keys=True)